In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

# Charger les données d'entraînement et de test
train_data = pd.read_csv('trainm.csv')
test_data = pd.read_csv('testm.csv')

# Séparer les features et la cible dans les données d'entraînement
X_train = train_data.drop(columns=['Exited'])  # Features
y_train = train_data['Exited']  # Cible

# Prétraitement des données
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns

# Créer un preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),  # Standardisation des features numériques
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)  # Encodage One-Hot des features catégorielles
    ])

# Modèle
model = LogisticRegression(max_iter=1000)  # Initialisation du modèle de régression logistique

# Créer un pipeline qui combine le prétraitement et le modèle
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', model)])

# Entraîner le modèle
pipeline.fit(X_train, y_train)

# Prétraitement des données de test
X_test = test_data

# Prédire la probabilité de 'Exited' pour les données de test
test_probabilities = pipeline.predict_proba(X_test)[:, 1]  # Probabilités de classe positive

# Créer un DataFrame pour les probabilités prédites
submission_df = pd.DataFrame({'id': test_data['id'], 'Exited': test_probabilities})

# Enregistrer les prédictions au format CSV
submission_df.to_csv('mysample_submission.csv', index=False)
